In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType
from pyspark.ml import Pipeline

In [3]:
#Iniciar a sessao Spark
spark = SparkSession.builder \
    .appName("DiabetesPrediction") \
    .getOrCreate()

# Carregar o dataset
df = spark.read.csv("pesquisa_diabetes.csv", header=True, inferSchema=True)

df.show()

+---+--------------------+-----+-----+----------------+--------+-----------------+-------------+
| id|          data_exame|idade|  imc|pressao_arterial|insulina|exercicio_semanal|nivel_glicose|
+---+--------------------+-----+-----+----------------+--------+-----------------+-------------+
|  1|2025-05-22 20:48:...|   56|33.86|             147|   53.91|                1|        83.43|
|  2|2025-03-26 20:48:...|   69|23.61|             149|  113.76|                3|        67.09|
|  3|2024-08-16 20:48:...|   46|32.77|             110|   44.32|                3|       168.76|
|  4|2024-11-28 20:48:...|   32|25.12|             117|  129.28|                3|       156.02|
|  5|2024-08-02 20:48:...|   60|23.53|             123|   52.98|                3|        65.14|
|  6|2024-12-30 20:48:...|   25|27.55|              97|   99.15|                1|       147.46|
|  7|2025-07-03 20:48:...|   78|28.71|             148|   70.14|                1|        88.02|
|  8|2024-12-28 20:48:...|   3

In [4]:
#Mostrar as primeiras linhas do DataFrame
print("Primeiras linhas do DataFrame:")
df.show(5)
print("\nEstrutura do DataFrame:")
df.printSchema()

Primeiras linhas do DataFrame:
+---+--------------------+-----+-----+----------------+--------+-----------------+-------------+
| id|          data_exame|idade|  imc|pressao_arterial|insulina|exercicio_semanal|nivel_glicose|
+---+--------------------+-----+-----+----------------+--------+-----------------+-------------+
|  1|2025-05-22 20:48:...|   56|33.86|             147|   53.91|                1|        83.43|
|  2|2025-03-26 20:48:...|   69|23.61|             149|  113.76|                3|        67.09|
|  3|2024-08-16 20:48:...|   46|32.77|             110|   44.32|                3|       168.76|
|  4|2024-11-28 20:48:...|   32|25.12|             117|  129.28|                3|       156.02|
|  5|2024-08-02 20:48:...|   60|23.53|             123|   52.98|                3|        65.14|
+---+--------------------+-----+-----+----------------+--------+-----------------+-------------+
only showing top 5 rows

Estrutura do DataFrame:
root
 |-- id: integer (nullable = true)
 |-- da

In [7]:
#Preparar os dados para o modelo

feature_columns = ["idade","imc", "pressao_arterial", "insulina", "exercicio_semanal"]  
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

#Criar e treinar o modelo de Random Forest
rf = RandomForestRegressor(featuresCol="scaledFeatures", labelCol="nivel_glicose", numTrees=100)

#criar um pipeline
pipeline = Pipeline(stages=[assembler, scaler, rf])

# Dividir os dados em treino e teste
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Ajustar o pipeline aos dados de treino
model = pipeline.fit(train_data)



In [8]:
#Fazer previsões nos dados de teste
predictions = model.transform(test_data)

# Avaliar o modelo
'''
    O erro quadrático médio (RMSE - Root Mean Square Error) é uma métrica comum para avaliar modelos de regressão.
    - O RMSE mede a média das diferenças quadráticas entre os valores previstos e os valores reais.
    - Um RMSE menor indica um modelo melhor, pois significa que as previsões estão mais próximas dos valores reais.
    - O RMSE é especialmente útil quando as diferenças entre os valores previstos e reais são importantes para a aplicação, como em previsões de saúde.
    - UM RMSE de 0.0 indica que o modelo prevê perfeitamente os valores reais, enquanto valores maiores indicam maior erro nas previsões.

'''

evaluator = RegressionEvaluator(labelCol="nivel_glicose", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Square Error (RMSE): {rmse}")





Root Mean Square Error (RMSE): 31.63000238934952


In [9]:
# Função para prever o nível de glicose de um novo paciente
def prever_nivel_glicose(idade, imc, pressao_arterial, insulina, exercicio_semanal):
    # Criar um DataFrame com os dados do novo paciente
    schema = StructType([
        StructField("idade", IntegerType(), True),
        StructField("imc", DoubleType(), True),
        StructField("pressao_arterial", IntegerType(), True),
        StructField("insulina", DoubleType(), True),
        StructField("exercicio_semanal", IntegerType(), True)
    ])
    
    novo_paciente = spark.createDataFrame([( idade, imc, pressao_arterial, insulina, exercicio_semanal)], schema)
    
    # Fazer previsões
    previsao = model.transform(novo_paciente)
    
    # Retornar o nível de glicose previsto
    return previsao.select("prediction").collect()[0][0]

In [10]:
# Exemplo de uso: Prever o nível de glicose para um novo paciente

idade = 45
imgc = 28.5
pressao_arterial = 130
insulina = 150.0
exercicio_semanal = 3


nivel_glicose_previsto = prever_nivel_glicose(idade, imgc, pressao_arterial, insulina, exercicio_semanal)

print(f"\n Previsão para o novo paciente:")
print(f"Idade: {idade}, IMC: {imgc}, Pressão Arterial: {pressao_arterial}, Insulina: {insulina}, Exercício Semanal: {exercicio_semanal}")
print(f"Nível de Glicose Previsto: {nivel_glicose_previsto:.2f}")


 Previsão para o novo paciente:
Idade: 45, IMC: 28.5, Pressão Arterial: 130, Insulina: 150.0, Exercício Semanal: 3
Nível de Glicose Previsto: 98.61
